In [2]:
from google.colab import files
uploaded = files.upload()

Saving orders.csv to orders.csv
Saving restaurants.sql to restaurants.sql
Saving users.json to users (1).json


In [3]:
!pip install pandas sqlalchemy


In [4]:
import pandas as pd
import sqlite3

In [7]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(':memory:')

with open('restaurants.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

conn.executescript(sql_script)


try:
    df_restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
    print("Success! Table 'restaurants' found.")
except Exception as e:
    print("Table 'restaurants' not found →", e)


tables = pd.read_sql_query("""
    SELECT name FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
""", conn)
print("\nAvailable tables:\n", tables)



conn.close()

Success! Table 'restaurants' found.

Available tables:
           name
0  restaurants


In [8]:
df_final = df_orders.merge(
    df_users,
    how='left',
    left_on='user_id',
    right_on='user_id'
).merge(
    df_restaurants,
    how='left',
    left_on='restaurant_id',
    right_on='restaurant_id'
)

print("Final dataset shape:", df_final.shape)
print("\nColumns in final dataset:\n", df_final.columns.tolist())
print("\nMissing values summary:\n", df_final.isna().sum())

df_final.head(3)

Final dataset shape: (10000, 12)

Columns in final dataset:
 ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']

Missing values summary:
 order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0


In [9]:
df_users = df_users.rename(columns={'id': 'user_id'})
df_restaurants = df_restaurants.rename(columns={'resto_id': 'restaurant_id'})

In [10]:
df_final.to_csv('final_food_delivery_dataset.csv', index=False)

from google.colab import files
files.download('final_food_delivery_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import pandas as pd


df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Columns:", df.columns.tolist())
print("\nMembership distribution:\n", df['membership'].value_counts(dropna=False))

gold_df = df[df['membership'].str.contains('Gold', case=False, na=False)]

revenue_by_city = (
    gold_df.groupby('city', as_index=False)['total_amount']
          .sum()
          .sort_values('total_amount', ascending=False)
          .reset_index(drop=True)
)

print("\nTop cities by total revenue from Gold members:")
print(revenue_by_city.head(10))

if not revenue_by_city.empty:
    top_city = revenue_by_city.iloc[0]
    print(f"\nCity with the **highest total revenue from Gold members**: **{top_city['city']}**")
    print(f"Total revenue: ₹{top_city['total_amount']:,.2f}")
else:
    print("No orders from Gold members found in the data.")

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']

Membership distribution:
 membership
Regular    5013
Gold       4987
Name: count, dtype: int64

Top cities by total revenue from Gold members:
        city  total_amount
0    Chennai    1080909.79
1       Pune    1003012.32
2  Bangalore     994702.59
3  Hyderabad     896740.19

City with the **highest total revenue from Gold members**: **Chennai**
Total revenue: ₹1,080,909.79


In [13]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Columns:", df.columns.tolist())
print("\nCuisine distribution:\n", df['cuisine'].value_counts(dropna=False))
print("\nTotal amount stats:\n", df['total_amount'].describe())

aov_by_cuisine = (
    df.groupby('cuisine', as_index=False)['total_amount']
      .mean()
      .rename(columns={'total_amount': 'avg_order_value'})
      .sort_values('avg_order_value', ascending=False)
      .round(2)
      .reset_index(drop=True)
)

print("\nAverage Order Value by Cuisine (highest to lowest):")
print(aov_by_cuisine)

if not aov_by_cuisine.empty:
    top = aov_by_cuisine.iloc[0]
    print(f"\nCuisine with the **highest average order value** across all orders: **{top['cuisine']}**")
    print(f"Average order value: ₹{top['avg_order_value']:,.2f}")
else:
    print("No data available – check if 'cuisine' and 'total_amount' columns exist.")

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']

Cuisine distribution:
 cuisine
Mexican    2581
Italian    2532
Indian     2469
Chinese    2418
Name: count, dtype: int64

Total amount stats:
 count    10000.000000
mean       801.162412
std        405.458753
min        100.200000
25%        446.310000
50%        806.295000
75%       1149.227500
max       1499.830000
Name: total_amount, dtype: float64

Average Order Value by Cuisine (highest to lowest):
   cuisine  avg_order_value
0  Mexican           808.02
1  Italian           799.45
2   Indian           798.47
3  Chinese           798.39

Cuisine with the **highest average order value** across all orders: **Mexican**
Average order value: ₹808.02


In [14]:
import pandas as pd


df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Columns:", df.columns.tolist())
print("\nShape of dataset:", df.shape)
print("Number of unique users:", df['user_id'].nunique())
print("\nTotal amount stats:", df['total_amount'].describe())

user_total_spend = (
    df.groupby('user_id', as_index=False)['total_amount']
      .sum()
      .rename(columns={'total_amount': 'total_spend'})
)

high_value_users = user_total_spend[user_total_spend['total_spend'] > 1000]

count = len(high_value_users)
distinct_count = high_value_users['user_id'].nunique()

print(f"\nNumber of distinct users who placed orders worth more than ₹1000 in total: **{count}**")
print(f"(Cross-check distinct count: {distinct_count})")


print("\nPercentage of users who are high-value (> ₹1000 total):")
print(f"{len(high_value_users) / df['user_id'].nunique() * 100:.1f}%")

print("\nTop 5 highest-spending users:")
print(
    user_total_spend.sort_values('total_spend', ascending=False)
                    .head(5)
                    .round(2)
)

print("\nDistribution of user total spend (quick bins):")
print(pd.cut(user_total_spend['total_spend'],
             bins=[0, 500, 1000, 2000, 5000, 10000, float('inf')],
             labels=['≤500', '501–1000', '1001–2000', '2001–5000', '5001–10000', '>10000'])
      .value_counts().sort_index())

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']

Shape of dataset: (10000, 12)
Number of unique users: 2883

Total amount stats: count    10000.000000
mean       801.162412
std        405.458753
min        100.200000
25%        446.310000
50%        806.295000
75%       1149.227500
max       1499.830000
Name: total_amount, dtype: float64

Number of distinct users who placed orders worth more than ₹1000 in total: **2544**
(Cross-check distinct count: 2544)

Percentage of users who are high-value (> ₹1000 total):
88.2%

Top 5 highest-spending users:
      user_id  total_spend
1451     1515     11556.49
627       650     10747.44
482       496      9634.30
2487     2586      9486.61
2516     2615      9237.32

Distribution of user total spend (quick bins):
total_spend
≤500           114
501–1000       225
1001–2000      699
2001–5000     1560
5001–10000     283
>100

In [15]:
import pandas as pd
import numpy as np

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Columns:", df.columns.tolist())
print("\nRating column summary:")
print(df['rating'].describe(include='all'))
print("\nUnique rating values:", sorted(df['rating'].dropna().unique()))
print("\nRating value counts:\n", df['rating'].value_counts(dropna=False))

df['rating_num'] = pd.to_numeric(df['rating'], errors='coerce')

invalid_count = df['rating_num'].isna().sum()
print(f"\nNumber of rows with invalid/non-numeric rating: {invalid_count}")
if invalid_count > 0:
    print("Sample invalid rows:\n", df[df['rating_num'].isna()][['restaurant_name_y', 'rating', 'total_amount']].head())

df_valid = df.dropna(subset=['rating_num'])

bins = [0, 3.0, 3.5, 4.0, 4.25, 4.5, 4.75, 5.0, np.inf]
labels = ['<3.0', '3.0–3.49', '3.5–3.99', '4.0–4.24', '4.25–4.49', '4.5–4.74', '4.75–5.0', '5.0+']

df_valid['rating_range'] = pd.cut(
    df_valid['rating_num'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=False
)

revenue_by_range = (
    df_valid.groupby('rating_range', observed=True)['total_amount']
           .sum()
           .reset_index()
           .sort_values('total_amount', ascending=False)
           .round(2)
)

revenue_by_range['total_amount_formatted'] = revenue_by_range['total_amount'].apply(lambda x: f"₹{x:,.2f}")

print("\nTotal Revenue by Restaurant Rating Range (highest to lowest):")
print(revenue_by_range[['rating_range', 'total_amount_formatted']])

if not revenue_by_range.empty:
    top_range = revenue_by_range.iloc[0]
    print(f"\nRestaurant rating range with the **highest total revenue**: **{top_range['rating_range']}**")
    print(f"Total revenue: {top_range['total_amount_formatted']}")
else:
    print("No valid rating data found — check the 'rating' column.")

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']

Rating column summary:
count    10000.000000
mean         4.045430
std          0.606531
min          3.000000
25%          3.500000
50%          4.100000
75%          4.600000
max          5.000000
Name: rating, dtype: float64

Unique rating values: [np.float64(3.0), np.float64(3.1), np.float64(3.2), np.float64(3.3), np.float64(3.4), np.float64(3.5), np.float64(3.6), np.float64(3.7), np.float64(3.8), np.float64(3.9), np.float64(4.0), np.float64(4.1), np.float64(4.2), np.float64(4.3), np.float64(4.4), np.float64(4.5), np.float64(4.6), np.float64(4.7), np.float64(4.8), np.float64(4.9), np.float64(5.0)]

Rating value counts:
 rating
4.8    831
4.9    608
4.6    605
4.5    602
3.2    599
3.8    562
3.1    548
4.7    518
4.2    512
4.1    486
3.7    466
3.4    438
4.4    422
4.3    409
3.5    396
4.0    391
3.9    387


In [16]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Columns:", df.columns.tolist())
print("\nMembership distribution:\n", df['membership'].value_counts(dropna=False))
print("\nNumber of Gold orders:", len(df[df['membership'].str.contains('Gold', case=False, na=False)]))
print("\nCity distribution (top 10):\n", df['city'].value_counts().head(10))

gold_df = df[df['membership'].str.contains('Gold', case=False, na=False)]

aov_by_city = (
    gold_df.groupby('city', as_index=False)['total_amount']
          .agg(['mean', 'count', 'sum'])
          .rename(columns={'mean': 'avg_order_value', 'count': 'num_orders', 'sum': 'total_revenue'})
          .round(2)
          .sort_values('avg_order_value', ascending=False)
          .reset_index(drop=True)
)

print("\nAverage Order Value (AOV) by City — Gold Members Only (highest to lowest):")
print(aov_by_city)

if not aov_by_city.empty:
    top_city = aov_by_city.iloc[0]
    print(f"\nAmong Gold members, the city with the **highest average order value** is: **{top_city['city']}**")
    print(f"Average order value: ₹{top_city['avg_order_value']:,.2f}")
    print(f"Based on {top_city['num_orders']} orders")
    print(f"Total revenue from Gold in this city: ₹{top_city['total_revenue']:,.2f}")
else:
    print("No Gold member orders found — double-check the 'membership' column values.")

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating', 'rating_num']

Membership distribution:
 membership
Regular    5013
Gold       4987
Name: count, dtype: int64

Number of Gold orders: 4987

City distribution (top 10):
 city
Bangalore    2751
Chennai      2469
Pune         2430
Hyderabad    2350
Name: count, dtype: int64

Average Order Value (AOV) by City — Gold Members Only (highest to lowest):
        city  avg_order_value  num_orders  total_revenue
0    Chennai           808.46        1337     1080909.79
1  Hyderabad           806.42        1112      896740.19
2  Bangalore           793.22        1254      994702.59
3       Pune           781.16        1284     1003012.32

Among Gold members, the city with the **highest average order value** is: **Chennai**
Average order value: ₹808.46
Based on 1337 orders
Total revenue from Gold in this city: ₹1,080,909.79


In [18]:
import pandas as pd

df = pd.read_csv('final_food_delivery_dataset.csv')

cuisine_stats = df.groupby('cuisine').agg(
    num_restaurants=('restaurant_id', 'nunique'),
    num_orders=('order_id', 'count'),
    total_revenue=('total_amount', 'sum'),
    avg_order_value=('total_amount', 'mean')
).reset_index()


cuisine_stats['revenue_per_restaurant'] = (
    cuisine_stats['total_revenue'] / cuisine_stats['num_restaurants']
).round(2)

result = cuisine_stats.sort_values(
    by=['num_restaurants', 'total_revenue'],
    ascending=[True, False]
).round(2)

print("Cuisines ranked by fewest restaurants, then highest revenue:")
print(result[['cuisine', 'num_restaurants', 'total_revenue',
              'revenue_per_restaurant', 'num_orders', 'avg_order_value']])

top_candidate = result.iloc[0]
print("\nBest match: cuisine with lowest # restaurants but significant revenue")
print(f"Cuisine:              {top_candidate['cuisine']}")
print(f"Distinct restaurants: {top_candidate['num_restaurants']}")
print(f"Total revenue:        ₹{top_candidate['total_revenue']:,.2f}")
print(f"Revenue per restaurant: ₹{top_candidate['revenue_per_restaurant']:,.2f}")
print(f"Number of orders:     {top_candidate['num_orders']}")
print(f"Avg order value:      ₹{top_candidate['avg_order_value']:,.2f}")

Cuisines ranked by fewest restaurants, then highest revenue:
   cuisine  num_restaurants  total_revenue  revenue_per_restaurant  \
0  Chinese              120     1930504.65                16087.54   
2  Italian              126     2024203.80                16065.11   
1   Indian              126     1971412.58                15646.13   
3  Mexican              128     2085503.09                16292.99   

   num_orders  avg_order_value  
0        2418           798.39  
2        2532           799.45  
1        2469           798.47  
3        2581           808.02  

Best match: cuisine with lowest # restaurants but significant revenue
Cuisine:              Chinese
Distinct restaurants: 120
Total revenue:        ₹1,930,504.65
Revenue per restaurant: ₹16,087.54
Number of orders:     2418
Avg order value:      ₹798.39


In [19]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("Membership distribution:")
print(df['membership'].value_counts(dropna=False))
print("\nTotal orders in dataset:", len(df))

gold_mask = df['membership'].str.contains('Gold', case=False, na=False)
gold_orders = gold_mask.sum()

total_orders = len(df)

percentage = (gold_orders / total_orders) * 100 if total_orders > 0 else 0
rounded_percentage = round(percentage)

print(f"\nOrders by Gold members: {gold_orders:,}")
print(f"Total orders: {total_orders:,}")
print(f"Percentage: {percentage:.2f}%")
print(f"Rounded to nearest integer: **{rounded_percentage}%**")

Membership distribution:
membership
Regular    5013
Gold       4987
Name: count, dtype: int64

Total orders in dataset: 10000

Orders by Gold members: 4,987
Total orders: 10,000
Percentage: 49.87%
Rounded to nearest integer: **50%**


In [28]:
import pandas as pd


df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

restaurant_stats = df.groupby('restaurant_name_y').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

low_volume = restaurant_stats[restaurant_stats['total_orders'] < 20]

ranked = low_volume.sort_values('avg_order_value', ascending=False)

print("Top restaurants with < 20 orders, sorted by highest AOV:")
print(ranked[['restaurant_name_y', 'total_orders', 'avg_order_value']].head(10).round(2))


if not ranked.empty:
    winner = ranked.iloc[0]
    print("\nRestaurant with HIGHEST average order value (but < 20 orders):")
    print(f"→ {winner['restaurant_name_y']}")
    print(f"   Avg order value: ₹{winner['avg_order_value']:.2f}")
    print(f"   Total orders: {winner['total_orders']}")
else:
    print("No restaurants found with < 20 orders in the dataset.")

Top restaurants with < 20 orders, sorted by highest AOV:
    restaurant_name_y  total_orders  avg_order_value
216    Restaurant_294            13          1040.22
181    Restaurant_262            18          1029.47
475     Restaurant_77            12          1029.18
104    Restaurant_193            15          1026.31
467      Restaurant_7            16          1002.14
220    Restaurant_298            15           989.82
452     Restaurant_56            18           989.47
40     Restaurant_135            18           988.70
271    Restaurant_343            17           986.03
237    Restaurant_312            14           982.73

Restaurant with HIGHEST average order value (but < 20 orders):
→ Restaurant_294
   Avg order value: ₹1040.22
   Total orders: 13


In [24]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

df['membership'] = df['membership'].astype(str).str.strip()

combinations = {
    'Gold + Indian cuisine': df[(df['membership'].str.contains('Gold', case=False, na=False)) &
                                (df['cuisine'] == 'Indian')]['total_amount'].sum(),

    'Gold + Italian cuisine': df[(df['membership'].str.contains('Gold', case=False, na=False)) &
                                 (df['cuisine'] == 'Italian')]['total_amount'].sum(),

    'Regular + Indian cuisine': df[(df['membership'].str.contains('Regular', case=False, na=False)) &
                                   (df['cuisine'] == 'Indian')]['total_amount'].sum(),

    'Regular + Chinese cuisine': df[(df['membership'].str.contains('Regular', case=False, na=False)) &
                                    (df['cuisine'] == 'Chinese')]['total_amount'].sum()
}

rev_df = pd.DataFrame(list(combinations.items()), columns=['Combination', 'Total Revenue'])
rev_df['Total Revenue'] = rev_df['Total Revenue'].round(2)
rev_df = rev_df.sort_values('Total Revenue', ascending=False).reset_index(drop=True)

print("Revenue by Combination (highest to lowest):")
print(rev_df.to_string(index=False))

winner = rev_df.iloc[0]
print(f"\nThe combination that contributes the **highest revenue** is:")
print(f"**{winner['Combination']}**")
print(f"Total revenue: ₹{winner['Total Revenue']:,.2f}")

Revenue by Combination (highest to lowest):
              Combination  Total Revenue
   Gold + Italian cuisine     1005779.05
 Regular + Indian cuisine      992100.27
    Gold + Indian cuisine      979312.31
Regular + Chinese cuisine      952790.91

The combination that contributes the **highest revenue** is:
**Gold + Italian cuisine**
Total revenue: ₹1,005,779.05


In [25]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

df = df.dropna(subset=['order_date'])

df['quarter'] = df['order_date'].dt.quarter

quarter_map = {
    1: 'Q1 (Jan–Mar)',
    2: 'Q2 (Apr–Jun)',
    3: 'Q3 (Jul–Sep)',
    4: 'Q4 (Oct–Dec)'
}

revenue_by_quarter = (
    df.groupby('quarter')['total_amount']
      .sum()
      .round(2)
      .rename(index=quarter_map)
      .sort_values(ascending=False)
)

orders_by_quarter = df.groupby('quarter').size().rename(index=quarter_map)

result_df = pd.DataFrame({
    'Total Revenue (₹)': revenue_by_quarter,
    'Number of Orders': orders_by_quarter
})

print("Revenue and Orders by Quarter:")
print(result_df)

highest_quarter = revenue_by_quarter.idxmax()
highest_revenue = revenue_by_quarter.max()

print(f"\nThe quarter with the **highest total revenue** is: **{highest_quarter}**")
print(f"Total revenue in that quarter: ₹{highest_revenue:,.2f}")

Revenue and Orders by Quarter:
              Total Revenue (₹)  Number of Orders
quarter                                          
Q1 (Jan–Mar)         2010626.64              2519
Q2 (Apr–Jun)         1945348.72              2440
Q3 (Jul–Sep)         2037385.10              2522
Q4 (Oct–Dec)         2018263.66              2519

The quarter with the **highest total revenue** is: **Q3 (Jul–Sep)**
Total revenue in that quarter: ₹2,037,385.10


/tmp/ipython-input-3610054084.py:5: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


In [29]:
import pandas as pd
df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')


print("Unique membership values:", df['membership'].unique())
print("\nMembership distribution:\n", df['membership'].value_counts(dropna=False))
gold_orders_count = df[df['membership'].str.contains('Gold', case=False, na=False)].shape[0]

print(f"\nTotal orders placed by Gold members: **{gold_orders_count:,}**")

Unique membership values: ['Regular' 'Gold']

Membership distribution:
 membership
Regular    5013
Gold       4987
Name: count, dtype: int64

Total orders placed by Gold members: **4,987**


In [30]:
import pandas as pd
df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')

print("All unique city names in the dataset:")
print(sorted(df['city'].dropna().unique()))

print("\nCity value counts (top 10):")
print(df['city'].value_counts().head(10))


hyderabad_exact = df[df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"\nRevenue (exact 'Hyderabad'): ₹{round(hyderabad_exact):,}")


hyderabad_any = df[df['city'].str.contains('Hyderabad', case=False, na=False)]['total_amount'].sum()
print(f"Revenue (any row containing 'Hyderabad'): ₹{round(hyderabad_any):,}")


if 'Hyderabad' in df['city'].values:
    final_revenue = hyderabad_exact
    print(f"\n**Total revenue from Hyderabad (rounded to nearest integer): ₹{round(final_revenue)}**")
else:
    print("\nNo exact 'Hyderabad' found — use the partial match or check spelling in cities above.")

All unique city names in the dataset:
['Bangalore', 'Chennai', 'Hyderabad', 'Pune']

City value counts (top 10):
city
Bangalore    2751
Chennai      2469
Pune         2430
Hyderabad    2350
Name: count, dtype: int64

Revenue (exact 'Hyderabad'): ₹1,889,367
Revenue (any row containing 'Hyderabad'): ₹1,889,367

**Total revenue from Hyderabad (rounded to nearest integer): ₹1889367**


In [31]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')


distinct_users = df['user_id'].nunique()


distinct_users_alt = df['user_id'].unique().size

print(f"Number of distinct users who placed at least one order: **{distinct_users}**")


print(f"Total number of orders: {len(df):,}")
print(f"Average orders per user: {len(df) / distinct_users:.2f}")

Number of distinct users who placed at least one order: **2883**
Total number of orders: 10,000
Average orders per user: 3.47


In [32]:
import pandas as pd

df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')
gold_orders = df[df['membership'].str.contains('Gold', case=False, na=False)]
aov_gold = round(gold_orders['total_amount'].mean(), 2)
print("Number of Gold orders:", len(gold_orders))
print("Total amount from Gold orders:", gold_orders['total_amount'].sum())
print(f"Average Order Value for Gold members: **₹{aov_gold}**")

Number of Gold orders: 4987
Total amount from Gold orders: 3975364.89
Average Order Value for Gold members: **₹797.15**


In [33]:
import pandas as pd


df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')


df['rating_num'] = pd.to_numeric(df['rating'], errors='coerce')


print("Rating column summary:")
print(df['rating'].describe(include='all'))
print("\nUnique rating values:", sorted(df['rating'].dropna().unique()))
print("\nMissing ratings:", df['rating_num'].isna().sum())
print("Total orders:", len(df))


high_rated_orders = df[df['rating_num'] >= 4.5]
count = len(high_rated_orders)


print(f"\nNumber of orders for restaurants with rating ≥ 4.5: **{count:,}**")
print(f"Percentage of total orders: {count / len(df) * 100:.1f}% (if total > 0)")


print("\nTop 5 restaurants with rating ≥ 4.5 by order count:")
print(
    high_rated_orders['restaurant_name_y']
    .value_counts()
    .head(5)
)

Rating column summary:
count    10000.000000
mean         4.045430
std          0.606531
min          3.000000
25%          3.500000
50%          4.100000
75%          4.600000
max          5.000000
Name: rating, dtype: float64

Unique rating values: [np.float64(3.0), np.float64(3.1), np.float64(3.2), np.float64(3.3), np.float64(3.4), np.float64(3.5), np.float64(3.6), np.float64(3.7), np.float64(3.8), np.float64(3.9), np.float64(4.0), np.float64(4.1), np.float64(4.2), np.float64(4.3), np.float64(4.4), np.float64(4.5), np.float64(4.6), np.float64(4.7), np.float64(4.8), np.float64(4.9), np.float64(5.0)]

Missing ratings: 0
Total orders: 10000

Number of orders for restaurants with rating ≥ 4.5: **3,374**
Percentage of total orders: 33.7% (if total > 0)

Top 5 restaurants with rating ≥ 4.5 by order count:
restaurant_name_y
Restaurant_287    35
Restaurant_92     32
Restaurant_289    30
Restaurant_188    30
Restaurant_475    30
Name: count, dtype: int64


In [34]:
import pandas as pd


df = df_final if 'df_final' in globals() else pd.read_csv('final_food_delivery_dataset.csv')


gold_df = df[df['membership'].str.contains('Gold', case=False, na=False)]


revenue_by_city = (
    gold_df.groupby('city')['total_amount']
          .sum()
          .sort_values(ascending=False)
)


top_city = revenue_by_city.index[0]
top_revenue = revenue_by_city.iloc[0]


orders_in_top_city = gold_df[gold_df['city'] == top_city].shape[0]


print("Revenue from Gold members by city (top 5):")
print(revenue_by_city.head(5).round(2))

print(f"\nTop revenue city for Gold members: **{top_city}**")
print(f"Total revenue in this city (Gold): ₹{top_revenue:,.2f}")
print(f"Number of orders placed in the top revenue city (Gold members only): **{orders_in_top_city:,}**")

Revenue from Gold members by city (top 5):
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

Top revenue city for Gold members: **Chennai**
Total revenue in this city (Gold): ₹1,080,909.79
Number of orders placed in the top revenue city (Gold members only): **1,337**
